In [2]:
%load_ext autoreload
%autoreload 2
import qcodes as qc

# QCoDeS config

The Qcodes config module uses JSON files to store QCoDeS configuration.

The config file controls various options to QCoDeS such as the default location where data is stored and logging level of debug output. A default configuration is shipped with QCoDeS. As we will see below you can overwrite this configuration in several different ways to customize the configuration. For instance you may want to change the location of the `qcodes.dataset` experiments db. In the example below you should see that by default it is stored in `~/experiments.db` (`~` is shorthand for the users home folder) but you can see that in the current config I have changed it to be stored inside a sub folder of my home folder. 


Qcodes loads both the defaults and the active configuration at module import so you can directly inspect them

In [37]:
qc.config.current_config

{'core': {'loglevel': 'INFO',
  'file_loglevel': 'INFO',
  'default_fmt': 'data/{date}/#{counter}_{name}_{time}',
  'register_magic': True,
  'db_location': 'c:\\Users\\jenielse\\mymainfolder\\experiments.db',
  'db_debug': False},
 'gui': {'notebook': True,
  'plotlib': 'all',
  'pyqtmaxplots': 100,
  'defaultcolormap': 'hot'},
 'user': {'scriptfolder': 'c:\\Users\\jenielse\\myscripts\\',
  'mainfolder': 'c:\\Users\\jenielse\\mymainfolder\\'},
 'station_configurator': {'enable_forced_reconnect': False,
  'default_folder': 'c:\\Users\\jenielse\\mymainfolder\\',
  'default_file': 'instrument_config.yaml'}}

In [38]:
qc.config.defaults

{'core': {'loglevel': 'WARNING',
  'file_loglevel': 'INFO',
  'default_fmt': 'data/{date}/#{counter}_{name}_{time}',
  'register_magic': True,
  'db_location': '~/experiments.db',
  'db_debug': False},
 'gui': {'notebook': True,
  'plotlib': 'all',
  'pyqtmaxplots': 100,
  'defaultcolormap': 'hot'},
 'user': {'scriptfolder': '.', 'mainfolder': '.'},
 'station_configurator': {'enable_forced_reconnect': False,
  'default_folder': '.',
  'default_file': 'instrument_config.yml'}}

One can inspect what the configuration options mean at runtime

In [16]:
print(qc.config.describe('core'))

controls core settings of qcodes.
Current value: {'loglevel': 'YOLO', 'file_loglevel': 'INFO', 'default_fmt': 'data/{date}/#{counter}_{name}_{time}', 'register_magic': True, 'db_location': 'c:\\Users\\jenielse\\mymainfolder\\experiments.db', 'db_debug': False}. Type: object. Default: Not defined.


## Configuring qcodes

Defaults are the settings that are shipped with the package, which you can overwrite programmatically.

A way to customize qcodes is to write your own json files, they are expected to be in the directories printed below.
One will be empty because one needs to define first the environment variable in the OS.

They are ordered by "weight", meaning that the last file always wins if it's overwriting any preconfigured defaults or values in the other files.

Simply copy the file to the directories and you are good to go.

In [18]:
print("\n".join([qc.config.home_file_name, qc.config.env_file_name, qc.config.cwd_file_name]))

C:\Users\jenielse\qcodesrc.json

C:\Users\jenielse\source\repos\Qcodes\docs\examples\qcodesrc.json


But the easiest way to add something to the config is to use the provided helper:

In [7]:
qc.config.add("base_location", "/dev/random", value_type="string", description="Location of data", default="/dev/random")

This will add a base_location with value /dev/random to the current config, and validate it's value to be of type string, will also set the description and what one would want to have as default.
The new entry is saved in the 'user' part of the config.

In [8]:
print(qc.config.describe('user.base_location'))

Location of data.
Current value: /dev/random. Type: string. Default: /dev/random.


You can also manually update the config from a specific file by suppling the path to the directory directly to `qc.config.update_config` as shown below. 

In [28]:
qc.config.update_config(path="C:\\Users\\jenielse\\")

{'core': {'loglevel': 'WARNING',
  'file_loglevel': 'INFO',
  'default_fmt': 'data/{date}/#{counter}_{name}_{time}',
  'register_magic': True,
  'db_location': 'c:\\Users\\jenielse\\mymainfolder\\experiments.db',
  'db_debug': False},
 'gui': {'notebook': True,
  'plotlib': 'all',
  'pyqtmaxplots': 100,
  'defaultcolormap': 'hot'},
 'user': {'scriptfolder': 'c:\\Users\\jenielse\\myscripts\\',
  'mainfolder': 'c:\\Users\\jenielse\\mymainfolder\\'},
 'station_configurator': {'enable_forced_reconnect': False,
  'default_folder': 'c:\\Users\\jenielse\\mymainfolder\\',
  'default_file': 'instrument_config.yaml'}}

## Saving changes

All the changes made to the defaults are stored, and one can then decide to save them to the expected place.

In [9]:
help(qc.config.save_to_cwd)

Help on method save_to_cwd in module qcodes.config.config:

save_to_cwd() method of qcodes.config.config.Config instance
    Save files to current working dir



In [10]:
help(qc.config.save_to_env)

Help on method save_to_env in module qcodes.config.config:

save_to_env() method of qcodes.config.config.Config instance
    Save  files to env path



In [11]:
help(qc.config.save_to_home)

Help on method save_to_home in module qcodes.config.config:

save_to_home() method of qcodes.config.config.Config instance
    Save  files to home dir



### Using  a custom configured variable  in your experiment:

Simply get the value you have set before with dot notation.
For example:

In [12]:
loc_provider = qc.data.location.FormatLocation(fmt=qc.config.user.base_location)
qc.data.data_set.DataSet.location_provider=loc_provider  

##  Changing core

one can change the core values at runtime, but there is no guarantee that they are valid.
Changing core values will also shadow the config from being updated if the defaults are updated upsteam (i.e. in the repo). So use it with a care.

In [33]:
qc.config.current_config.core.loglevel = 'INFO'

But one can maunually validate 

In [31]:
qc.config.validate()

Which will raise an exception in case of bad inputs

In [32]:
qc.config.current_config.core.loglevel = 'YOLO'
qc.config.validate()
# NOTE that you how have a broken config! 

ValidationError: 'YOLO' is not one of ['CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG']

Failed validating 'enum' in schema['properties']['core']['properties']['loglevel']:
    {'default': 'DEBUG',
     'description': 'control logging  level',
     'enum': ['CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG'],
     'type': 'string'}

On instance['core']['loglevel']:
    'YOLO'